## ANN Regression Practical Implementation

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [2]:
data = pd.read_csv('../Datasets/Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
## Preprocess the data
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [4]:
## Define the label encoder
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [5]:
# One-hot encoder for the geography
onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [6]:
## Combining the one hot encoded geography to original data
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [7]:
# Split the data in features and target
X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

In [8]:
# Split the data in training and testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Standard Scaler
from sklearn.preprocessing import StandardScaler, LabelEncoder
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
#  Save the Encoders and scaler for user
with open('label_encoder_gender.pkl', 'wb') as file:
        pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
        pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
        pickle.dump(scaler, file)

### ANN Regression Problem Statement

In [11]:
X_train.shape[1],

(12,)

In [12]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_train shape: (8000, 12)
y_train shape: (8000,)
X_test shape: (2000, 12)
y_test shape: (2000,)


In [13]:
import tensorflow as tf
print(tf.__version__)


2.19.0


In [14]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime
# Setup of tensorboard
log_dir="regressionlogs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback=TensorBoard(log_dir=log_dir, histogram_freq=1)

In [15]:
## setup Early Stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

# Step 2: Ensure data is in correct format
X_train = np.array(X_train, dtype=np.float32)
X_test = np.array(X_test, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32).reshape(-1, 1)
y_test = np.array(y_test, dtype=np.float32).reshape(-1, 1)

# Step 3: Clear any previous TensorFlow models
tf.keras.backend.clear_session()

# Step 4: Build the model with correct layer names
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],), name='InputLayer'),
    Dense(32, activation='relu', name='HiddenLayer1'),
    Dense(1, name='OutputLayer')
])

model.summary()

# Step 5: Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

# Step 6: Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, callbacks=[early_stopping_callback, tensorboard_callback])


c:\Users\Uditya\Desktop\Deep Learning\Deep Learning for Beginner\venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ InputLayer (Dense)              │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ HiddenLayer1 (Dense)            │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ OutputLayer (Dense)             │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - loss: 100591.4062 - mae: 100591.4062 - val_loss: 98584.9141 - val_mae: 98584.9141
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 99912.1016 - mae: 99912.1016 - val_loss: 97516.2109 - val_mae: 97516.2109
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 98909.4766 - mae: 98909.4766 - val_loss: 94650.1797 - val_mae: 94650.1797
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 95055.3203 - mae: 95055.3203 - val_loss: 89670.6172 - val_mae: 89670.6172
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 89792.1484 - mae: 89792.1484 - val_loss: 82875.3984 - val_mae: 82875.3984
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 81031.4141 - mae: 81031.4141 - val_loss: 74977.4062 - val_mae: 74977.4062
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 74227.1250 - mae: 74227.1250 - val_loss: 67322.1406 - val_mae: 67322.1406
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms

In [17]:
%load_ext tensorboard 

In [25]:
%tensorboard --logdir regressionlogs/fit20250406-204615

Reusing TensorBoard on port 6007 (pid 5060), started 0:00:07 ago. (Use '!kill 5060' to kill it.)

In [19]:
## Evaluate the model on test data
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test_MAE {test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 51113.4570 - mae: 51113.4570
Test_MAE 50299.74609375


In [20]:
model.save('regression.h5')

###  By increasing of layers the model  learns better

In [21]:
from tensorflow.keras.layers import LeakyReLU

model = Sequential([
    Dense(128, input_shape=(X_train.shape[1],), name='InputLayer'),
    LeakyReLU(alpha=0.1),
    
    Dense(64, name='FirstHiddenLayer'),
    LeakyReLU(alpha=0.1),
    
    Dense(32,name='SecondHiddenLayer'), 
    LeakyReLU(alpha=0.1),
    
    Dense(1, name='OutputLayer')
])


c:\Users\Uditya\Desktop\Deep Learning\Deep Learning for Beginner\venv\lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [22]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ InputLayer (Dense)              │ (None, 128)            │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ FirstHiddenLayer (Dense)        │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SecondHiddenLayer (Dense)       │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ OutputLayer (Dense)             │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,033 (47.00 KB)

 Trainable params: 12,033 (47.00 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,  
    batch_size=32,
    callbacks=[early_stop]
)


Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 99269.6797 - mae: 99269.6797 - val_loss: 83595.3750 - val_mae: 83595.3750
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 71407.0469 - mae: 71407.0469 - val_loss: 50281.8477 - val_mae: 50281.8477
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 49550.1953 - mae: 49550.1953 - val_loss: 50326.9570 - val_mae: 50326.9570
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 50154.7031 - mae: 50154.7031 - val_loss: 50269.2188 - val_mae: 50269.2188
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 49554.1250 - mae: 49554.1250 - val_loss: 50260.6133 - val_mae: 50260.6133
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 49607.3203 - mae: 49607.3203 - val_loss: 50212.0781 - val_mae: 50212.0781
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 49838.6875 - mae: 49838.6875 - val_loss: 50270.5312 - val_mae: 50270.5312
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - 